In [5]:
# 데이터 처리 모듈
import pandas as pd
# 행렬 등 통계 연산 모듈
import numpy as np
# 지수형 표기법 e를 연속형 변환
pd.options.display.float_format = '{:.10f}'.format
# 타입 어노테이션(Any, Sequence 등의 메서드 활용)
from typing import *

# 구글 드라이브 마운트
from google.colab import drive
drive.mount("/content/drive")
import os

Mounted at /content/drive


In [87]:
class MergeData():

  def __init__(self, soho_credit_data = None):
    self.soho_credit_data = soho_credit_data

  def mergeSohoCredit(self, date_time: int, file_path: str, column_name_list: List):
    # date_time에 202201 ~ 202212를 입력값으로 받고, file_path에 해당 데이터가 있는 드라이브 경로 설정
    # file_path = # /content/drive/MyDrive/2023BigContest/Archive/"

    credit_data_list: List = []
      # 월별 데이터프레임을 담을 리스트 생성
    date_time_list = [str(date_time + n) for n in range(0, 12, 1)]
    for date in date_time_list:
      data = pd.read_csv(
          os.path.join(file_path + date + "_INCOME_230907.csv"),
          index_col = 0, sep = ",",
          names = column_name_list)
          # header = None)
      data.drop("index", axis = 0, inplace = True)
      data[column_name_list[1:]] = data[column_name_list[1:]].astype("float")

      # data["손익분기점_매출액_STD"] = data["월임대료"] / (1 - (data["부가가치세_STD"] + data["매입액"]) / V["매출액"]) = 고정비용 / (1-변동비율)
      data["객단가_STD"] = data["매출액_STD"] / data["고객수_STD"]
      data["신규고객단가_STD"] = data["매출액_STD"] / data["신규고객수_STD"]
      data["전체대비_주말매출액비중_STD"] = data["주말매출액_STD"] / data["매출액_STD"]
        # 파생컬럼 생성

      credit_data_list.append(data)
        # os.path.join을 통한 구글 드라이브 파일 내 경로로 csv데이터 로드하여 판다스 데이터프레임으로 저장 후
        # credit_data_list에 0 ~ 11번째 인덱스의 원소로 추가

    return credit_data_list
      # 2022년 12개월의 소호 상권별 신용거래정보 데이터프레임을 담은 리스트 반환
      # 병합하여 self.soho_credit_data라는 멤버변수에 저장할 것

In [88]:
Preprocess = MergeData()

In [89]:
credit_data_list = Preprocess.mergeSohoCredit(
    date_time = 202201,
    file_path = "/content/drive/MyDrive/2023BigContest/data/KCD_CREDIT_DATA/SOHO_",
    column_name_list = ["상권명", "매출액_STD", "매입액_STD", "매출총이익_STD", "부가가치세_STD", "부가세차감전영업이익_STD", "영업이익_STD", "주말매출액_STD", "고객수_STD", "신규고객수_STD"])

```
* standard_deviation_population(kcd.transactions.sum_sales_card + kcd.transactions.sum_sales_delivery)
* standard_deviation_population(kcd.transactions.sum_purchase_card + kcd.transactions.sum_purchase_cash)
* standard_deviation_population((kcd.transactions.sum_sales_card + kcd.transactions.sum_sales_delivery) - (kcd.transactions.sum_purchase_card + kcd.transactions.sum_purchase_cash))
* standard_deviation_population(kcd.transactions.sum_sales_invoice - kcd.transactions.sum_purchase_invoice)
* standard_deviation_population(((kcd.transactions.sum_sales_card + kcd.transactions.sum_sales_delivery) - (kcd.transactions.sum_purchase_card + kcd.transactions.sum_purchase_cash)) - kcd.transactions.monthly_rental_fee)
* standard_deviation_population((((kcd.transactions.sum_sales_card + kcd.transactions.sum_sales_delivery) - (kcd.transactions.sum_purchase_card + kcd.transactions.sum_purchase_cash)) - kcd.transactions.monthly_rental_fee) - (kcd.transactions.sum_sales_invoice - kcd.transactions.sum_purchase_invoice))
* standard_deviation_population(kcd.transactions.sum_weekend_sales_card + kcd.transactions.sum_weekend_sales_delivery)
* standard_deviation_population(sum_customer_cnt)
* standard_deviation_population(sum_new_customer_cnt)
```


In [96]:
display(credit_data_list[11].info())
# display(credit_data_list[11])

<class 'pandas.core.frame.DataFrame'>
Index: 93 entries, 0 to 92
Data columns (total 13 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   상권명               93 non-null     object 
 1   매출액_STD           93 non-null     float64
 2   매입액_STD           93 non-null     float64
 3   매출총이익_STD         93 non-null     float64
 4   부가가치세_STD         93 non-null     float64
 5   부가세차감전영업이익_STD    93 non-null     float64
 6   영업이익_STD          93 non-null     float64
 7   주말매출액_STD         93 non-null     float64
 8   고객수_STD           93 non-null     float64
 9   신규고객수_STD         93 non-null     float64
 10  객단가_STD           93 non-null     float64
 11  신규고객단가_STD        93 non-null     float64
 12  전체대비_주말매출액비중_STD  93 non-null     float64
dtypes: float64(12), object(1)
memory usage: 12.2+ KB


None